In [ ]:
!pip install roboflow

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

import torch
import cv2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from roboflow import Roboflow


In [ ]:

rf = Roboflow(api_key="RRJitllisZT37MLdkgjz")
project = rf.workspace("project-rt7v1").project("-b107j")
version = project.version(4)
dataset = version.download("coco")
                

In [10]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))

# Замените количество ключевых точек
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 4  # Важно!
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

# Заморозьте слои, если данных мало (опционально)
cfg.MODEL.BACKBONE.FREEZE_AT = 2  # Заморозить первые 2 слоя ResNet

In [11]:
# Регистрация кастомных метаданных
MetadataCatalog.get("my_dataset_train").keypoint_names = ["point1", "point2", "point3", "point4"]
MetadataCatalog.get("my_dataset_train").keypoint_flip_map = []  # Если точки несимметричны
MetadataCatalog.get("my_dataset_train").thing_classes = ["object"]  # Класс объекта

# Если есть связи между точками (например, точка0 → точка1)
MetadataCatalog.get("my_dataset_train").keypoint_skeleton = [[0,1], [1,2], [2,3]]

In [12]:
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4  # Размер батча
cfg.SOLVER.BASE_LR = 0.001  # Learning rate
cfg.SOLVER.MAX_ITER = 2000  # Количество итераций
cfg.TEST.EVAL_PERIOD = 500  # Оценка каждые 500 итераций

# Запустите обучение
from detectron2.engine import DefaultTrainer
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

AssertionError: Torch not compiled with CUDA enabled